In [ ]:
!pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
#make detections
cap = cv2.VideoCapture(0)
counter = 0
state = None
#setup mediapipe instance
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #making detection
        results = pose.process(image)
        
        #recolor image to BGR
        image.flags.writeable=True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            #get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            #calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            #visualize
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            #curlcounter algo
            if angle > 160:
                state = "down"
            if angle <30 and state =='down':
                state = "up"
                counter+=1
                print(counter)
            
        except:
            pass
        
        #visualize curlcounter
        cv2.rectangle(image, (0,0), (350,73), (255, 255, 255)) #start, end, color, line width
        
        cv2.putText(image, 'CURL COUNTS:', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (245,117,16), 2, cv2.LINE_AA)
        
        cv2.putText(image, 'STATE:', (155,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, state, (150,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (245,117,16), 2, cv2.LINE_AA)
        
        #render detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,290), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows() 

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a) #first
    b = np.array(b) #mid
    c = np.array(c) #end
    
    radian = np.arctan2(c[1]-b[1], c[0]-b[0])-np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radian*180.0/np.pi)
    
    if angle >180.0:
        angle = 360 - angle
        
    return angle